In [1]:
import ee
import geemap
import numpy as np
import sys
sys.path.insert(1, '/Users/gopal/Projects/ml/downloadGEErasters')
import rs
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from gee_landsat_prep_functions import prep_tm5_ic, prep_etm7_ic, prep_oli8_ic, prep_landsat_clouds

In [2]:
ee.Initialize()

In [3]:
ee_geometry_box = ee.Geometry.Polygon(
        [[[77.42201334186198, 11.398136848502652],
          [77.42201334186198, 10.873987098476817],
          [78.33387857623698, 10.873987098476817],
          [78.33387857623698, 11.398136848502652]]], None, False)

In [4]:
ee_geometry = ee.FeatureCollection("users/gopalpenny/cauvery/Cauvery_boundary5")


In [5]:
# Prepare and merge landsat data
tm5 = prep_tm5_ic(ee_geometry) 
etm7 = prep_etm7_ic(ee_geometry)
oli8 = prep_oli8_ic(ee_geometry) 
landsat = tm5.merge(etm7).merge(oli8) \
    .filterMetadata('CLOUD_COVER', 'less_than', 50)


In [6]:
landsat_clouds = prep_landsat_clouds(landsat)

In [7]:
def add_ndvi_band(im):
    im = im.addBands(im.normalizedDifference(['nir','red']).rename('NDVI')) 
    im = im.addBands(im.normalizedDifference(['green','swir1']).rename('MNDWI'))
    im = im.updateMask(im.select(['clouds_shadows']).eq(0))
    im = im.float()
    return im

landsat_bands = landsat_clouds.map(add_ndvi_band)
# landsat_bands.size().getInfo()


In [8]:
# landsat_bands.first().bandNames().getInfo()

['swir2',
 'swir1',
 'nir',
 'red',
 'green',
 'blue',
 'clouds',
 'shadows',
 'clouds_shadows',
 'NDVI',
 'MNDWI']

In [9]:
def season_median(ic, start_date, end_date, season_name):
    
    # print('start_date',start_date)
    # print('end_date',end_date)
    band_names_orig = ['nir','red','green','blue','NDVI','MNDWI']
    band_names_median = [name + '_median' for name in band_names_orig]
    band_names_season = [name + '_' + season_name for name in band_names_orig]
    band_names_max = [name + '_max' for name in band_names_orig]
    ic = ic \
      .filterDate(start_date, end_date)
    num_images = ic.size().getInfo()
    print(season_name + ' num images:',num_images)
    
    season_median = ic \
      .reduce(ee.Reducer.median()) \
      .select(band_names_median, band_names_season)
    season_max = ic \
      .reduce(ee.Reducer.max()) \
      .select(['NDVI_max'], ['NDVI_max_' + season_name])
    season_median = season_median \
      .addBands(ic.select(['NDVI']).reduce(ee.Reducer.count()).rename(season_name + '_count')) \
      .addBands(season_max)
    
    return season_median

In [10]:
def get_im_seasons(ic, monsoon_year, ndvi_threshold = 0.2):
    
    print('Starting monsoon year:', monsoon_year)
    
    # specify the dates for each of the seasons
    kharif_start_dt = datetime.strptime(str(monsoon_year) + "-06-01", "%Y-%m-%d")
    kharif_start_date = datetime.strftime(kharif_start_dt, "%Y-%m-%d")
    rabi_start_date = datetime.strftime(kharif_start_dt + relativedelta(months = 4), "%Y-%m-%d")
    summer_start_date = datetime.strftime(kharif_start_dt + relativedelta(months = 8), "%Y-%m-%d")
    summer_end_date = datetime.strftime(kharif_start_dt + relativedelta(months = 12), "%Y-%m-%d")

    # calculate the median bands for each of the seasons
    im_kharif = season_median(ic, kharif_start_date, rabi_start_date, 'kharif')
    im_rabi = season_median(ic, rabi_start_date, summer_start_date, 'rabi')
    im_summer = season_median(ic, summer_start_date, summer_end_date, 'summer')
    
    # combine the bands from each season to a single image
    im_seasons = im_kharif \
      .addBands(im_rabi) \
      .addBands(im_summer) \
      .set('monsoon_year', monsoon_year)
    
    im_seasons = im_seasons.addBands(
        im_seasons.select(['NDVI_kharif']).gt(ndvi_threshold).rename('kharif_crops')).addBands(
        im_seasons.select(['NDVI_rabi']).gt(ndvi_threshold).rename('rabi_crops')).addBands(
        im_seasons.select(['NDVI_summer']).gt(ndvi_threshold).rename('summer_crops')) #.addBands(
        # im_seasons.expression('(im.NDVI_kharif > 0.25) + (im.NDVI_rabi > 0.25) + (im.NDVI_summer > 0.25)', 
        #   {'im':im_seasons}).rename('crops'))
    im_seasons = im_seasons.addBands(
        im_seasons.select(['kharif_crops','rabi_crops','summer_crops']).reduce(ee.Reducer.sum()).rename('crops'))
        

    return im_seasons

In [11]:
# monsoon_year = 2015
# im_seasons_05 = get_im_seasons(landsat_bands, 2005)
# im_seasons_06 = get_im_seasons(landsat_bands, 2006)
# im_seasons_06.bandNames().getInfo()

In [15]:

# im_seasons_05_25 = get_im_seasons(landsat_bands, 2005, 0.25)
# im_seasons_06_25 = get_im_seasons(landsat_bands, 2006, 0.25)
# im_seasons_07_25 = get_im_seasons(landsat_bands, 2007, 0.25)

im_seasons_06_22 = get_im_seasons(landsat_bands, 2006, 0.22)

# im_seasons_05_20 = get_im_seasons(landsat_bands, 2005, 0.2)
# im_seasons_06_20 = get_im_seasons(landsat_bands, 2006, 0.2)
# im_seasons_07_20 = get_im_seasons(landsat_bands, 2007, 0.2)


# print(im_season)
# im_seasons_15 = get_im_seasons(landsat_bands, 2015)
# im_seasons_16 = get_im_seasons(landsat_bands, 2016)
# im_seasons_17 = get_im_seasons(landsat_bands, 2017)
# im_seasons_18 = get_im_seasons(landsat_bands, 2018)
# im_seasons_19 = get_im_seasons(landsat_bands, 2019)
# im_seasons_20 = get_im_seasons(landsat_bands, 2020)
# im_seasons_21 = get_im_seasons(landsat_bands, 2021)

Starting monsoon year: 2006
kharif num images: 30
rabi num images: 57
summer num images: 77


In [14]:
im_seasons_all = ee.ImageCollection([im_seasons_15, 
                                     im_seasons_16,
                                    #  im_seasons_17,
                                    #  im_seasons_18,
                                    #  im_seasons_19,
                                    #  im_seasons_20,
                                     im_seasons_21])


3

In [16]:

m16.addLayer(im_seasons_06_22.clip(ee_geometry), {'bands':['crops'], 'min':0, 'max':3, 'palette': colors},'crops 2006 NDVI>0.22')

In [15]:
def get_crops_bit_adj(im):
    monsoon_year = im.get('monsoon_year')
    crops_year_diff = ee.Number(monsoon_year).subtract(2015)
    crops_mult = ee.Number(4).pow(crops_year_diff)
    
    return im.select(['crops']).multiply(crops_mult)
    # crops_bit = 
    
    
crops_2bit_base2015 = im_seasons_all \
  .filter(ee.Filter.lte('monsoon_year',2015)) \
  .map(get_crops_bit_adj) \
  .reduce(ee.Reducer.sum())
    

In [16]:
def select_crops_year(im):
    monsoon_year = ee.String(im.get('monsoon_year'))
    band_name = ee.String('y').cat(monsoon_year)
    
    return im.select(['crops'],[band_name])
    

crops_bands = im_seasons_all.select(['crops']).toBands()

crops_bands.getInfo()

{'type': 'Image',
 'bands': [{'id': '0_crops',
   'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 3},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': '1_crops',
   'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 3},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': '2_crops',
   'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 3},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}]}

In [17]:
mall = geemap.Map()

mall.addLayer(crops_2bit_base2015,{'min':0,'max':16000},'crops_2bit_base2015')
mall.centerObject(ee_geometry_box, 10)
mall.addLayerControl()
mall

Map(center=[11.136097907071127, 77.87794595904944], controls=(WidgetControl(options=['position', 'transparent_…

In [18]:
crops_task = ee.batch.Export.image.toDrive(
    image = crops_bands,
    description = 'crops_bands',
    folder = 'classy_downloads',
    fileNamePrefix = 'karur_crops_2015_2021',
    region = ee_geometry_box,
    scale = 30)

# crops_task.start()

In [ ]:
ee.batch.Export.image

In [19]:
im_seasons_16 = im_seasons_16.addBands(
    im_seasons_16.expression('(im.NDVI_kharif > 0.25) + (im.NDVI_rabi > 0.25) + (im.NDVI_summer > 0.25)', {'im':im_seasons_16}).rename('crops')
)

im_seasons_16

In [ ]:
m.addLayer(oli8.first(), {'bands':['SR_B5','SR_B4','SR_B3'], 'min':5000, 'max':30000},'oli8')


In [24]:
im_seasons_05.getInfo()

{'type': 'Image',
 'bands': [{'id': 'nir_kharif',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'red_kharif',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'green_kharif',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'blue_kharif',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'NDVI_kharif',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'MNDWI_kharif',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'nir_rabi',
   'data_type': {'type': 'PixelType', 'precision': 'float'

In [53]:
# gfsad30 = ee.Image("users/gopalpenny/GFSAD30")
m = geemap.Map()

colors = ['#D2B48C',  '#EEC900', '#90EE90',  '#228B22']
colors7 = ["#F1F1F1","#E8DCCA","#B5C8A2","#68B094","#008E9A","#395E96","#6B0077"]

m.addLayer(im_seasons_05.clip(ee_geometry), {'bands':['crops'], 'min':0, 'max':3, 'palette': colors},'crops 2005')
m.addLayer(im_seasons_06.clip(ee_geometry), {'bands':['crops'], 'min':0, 'max':3, 'palette': colors},'crops 2006')

# m.addLayer(im_seasons_06.clip(ee_geometry), {'bands':['kharif_count'], 'min':0, 'max':6, 'palette': colors7},'kharif count 2006')
# m.addLayer(im_seasons_06.clip(ee_geometry), {'bands':['rabi_count'], 'min':0, 'max':6, 'palette': colors7},'rabi count 2006')
# m.addLayer(im_seasons_06.clip(ee_geometry), {'bands':['summer_count'], 'min':0, 'max':6, 'palette': colors7},'summer count 2006')
# m.addLayer(im_seasons_15.clip(ee_geometry), {'bands':['crops'], 'min':0, 'max':3, 'palette': colors},'crops 2015')
# m.addLayer(im_seasons_16.clip(ee_geometry), {'bands':['crops'], 'min':0, 'max':3, 'palette': colors},'crops 2016')
# m.addLayer(im_seasons_21.clip(ee_geometry), {'bands':['crops'], 'min':0, 'max':3, 'palette': colors},'crops 2021')
# m.centerObject(ee_geometry, 13)
m.addLayerControl()
m

Map(center=[11.748174793282523, 77.4497503999458], controls=(WidgetControl(options=['position', 'transparent_b…

In [14]:
m16 = geemap.Map()

colors = ['#D2B48C',  '#EEC900', '#90EE90',  '#228B22']

# m.addLayer(gfsad30, {'min':0, 'max':2},'gfsad30')
# m16.addLayer(im_seasons_16, {'bands':['SR_B5_kharif','SR_B4_kharif','SR_B3_kharif'], 'min':5000, 'max':30000},'oli8 kharif')
# m16.addLayer(im_seasons_16, {'bands':['SR_B5_rabi','SR_B4_rabi','SR_B3_rabi'], 'min':5000, 'max':30000},'oli8 rabi')
m16.addLayer(im_seasons_06_20.clip(ee_geometry), {'bands':['nir_kharif','red_kharif','green_kharif'], 'min':0, 'max':30000},'kharif 2006')
m16.addLayer(im_seasons_06_20.clip(ee_geometry), {'bands':['nir_kharif','red_rabi','green_rabi'], 'min':0, 'max':30000},'rabi 2006')
m16.addLayer(im_seasons_06_20.clip(ee_geometry), {'bands':['nir_kharif','red_summer','green_summer'], 'min':0, 'max':30000},'summer 2006')
m16.addLayer(im_seasons_06_20.clip(ee_geometry), {'bands':['crops'], 'min':0, 'max':3, 'palette': colors},'crops 2006 NDVI>0.2')
m16.addLayer(im_seasons_06_25.clip(ee_geometry), {'bands':['crops'], 'min':0, 'max':3, 'palette': colors},'crops 2006 NDVI>0.25')
m16.addLayer(im_seasons_06_22.clip(ee_geometry), {'bands':['crops'], 'min':0, 'max':3, 'palette': colors},'crops 2006 NDVI>0.22')
m16.addLayer(im_seasons_06_20.clip(ee_geometry), {'bands':['crops'], 'min':0, 'max':3, 'palette': colors},'crops 2006 NDVI>0.2')
# m16.addLayer(im_seasons_06.clip(ee_geometry), {'bands':['NDVI_rabi'], 'min':0, 'max':0.5},'ndvi rabi')
# m16.addLayer(im_seasons_06.clip(ee_geometry), {'bands':['NDVI_summer'], 'min':0, 'max':0.5},'ndvi summer')
# m16.addLayer(im_seasons_06.clip(ee_geometry), {'bands':['NDVI_kharif'], 'min':0, 'max':0.5},'ndvi kharif')
# m16.addLayer(im_seasons_06.clip(ee_geometry), {'bands':['NDVI_rabi'], 'min':0, 'max':0.5},'ndvi rabi')
# m16.addLayer(im_seasons_06.clip(ee_geometry), {'bands':['NDVI_summer'], 'min':0, 'max':0.5},'ndvi summer')
# m16.addLayer(im_seasons_06.clip(ee_geometry), {'bands':['NDVI_kharif','NDVI_rabi','NDVI_summer'], 'min':0, 'max':0.5},'ndvi all')

# m16.addLayer(im_seasons_16.gt(0.25), {'bands':['NDVI_rabi'], 'min':0, 'max':0.5},'crop rabi')
# m16.addLayer(im_seasons_15, {'bands':['crops'], 'min':0, 'max':3},'crops 2015')
# m16.addLayer(im_seasons_16, {'bands':['crops'], 'min':0, 'max':3},'crops 2016')
# m16.addLayer(im_seasons_16, {'bands':['crops'], 'min':0, 'max':3},'crops 2016')
# m16.addLayer(im_seasons_16, {'bands':['MNDWI_kharif','NDVI_rabi','NDVI_summer'], 'min':0.1, 'max':0.3},'mndwi kharif')
# m16.addLayer(im_seasons_16, {'bands':['MNDWI_kharif'], 'min':-0.2, 'max':0.1},'mndwi kharif')
# m16.addLayer(im_seasons_16, {'bands':['MNDWI_rabi'], 'min':-0.2, 'max':0.1},'mndwi rabi')
# m.addLayer(oli8_summer, {'bands':['NDVI'], 'min':0, 'max':0.5},'ndvi summer')
# m.addLayer(oli8_kharif, {'bands':['MNDWI'], 'min':-0.3, 'max':0},'ndwi_median')
# m.addLayer(ee_geometry)
# m16.centerObject(ee_geometry, 13)
m16.addLayerControl()
m16

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…